In [ ]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import os
import gc
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import transforms
from IPython.display import clear_output
import pandas as pd
import wandb
import random
import copy
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, CLIPProcessor, CLIPModel, AutoImageProcessor, AutoModel, AutoModelForCausalLM, AutoProcessor, VisionTextDualEncoderModel, VisionTextDualEncoderProcessor
from transformers import ViltProcessor, ViltModel
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
        

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device1 = "cuda:1" if torch.cuda.device_count() > 1 else "cpu"

print(f"Thiết bị 0: {device}")
print(f"Thiết bị 1: {device1}")


In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('nguyenduytamanh03012004-uit/huggingface/fine_tune_vit5:v0', type='model')
# artifact_dir = artifact.download()
# run.finish()

In [ ]:
# # vit5_finetune-----------------------------------------------------------
# # vit5_model = AutoModelForSeq2SeqLM.from_pretrained(artifact_dir)
# # vit5_processor = AutoTokenizer.from_pretrained(artifact_dir)
# # vit5 -------------------------------------------------------------------
# vit5_model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
# vit5_processor = AutoTokenizer.from_pretrained("VietAI/vit5-base")
# #clip ---------------------------------------------------------------------
# image_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
# clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# vilt_processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
# vilt_model = ViltModel.from_pretrained("dandelin/vilt-b32-mlm")
# # #dino --------------------------------------------------------------------
# # dino_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
# # dino_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
# # visocial_T5_fintune -------------------------------------------------------
# # visocial_processor = AutoTokenizer.from_pretrained("5CD-AI/visocial-T5-base")
# # visocial_model = AutoModelForSeq2SeqLM.from_pretrained(artifact_dir).to(device)
# # llava---------------------------------------------------------------------------
# # llava_model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
# # llava_processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
# # vilt

# # bert & vit
# # bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
# # vit_image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
# # processor = VisionTextDualEncoderProcessor(vit_image_processor, bert_tokenizer)
# # model = VisionTextDualEncoderModel.from_vision_text_pretrained(
# #     "google/vit-base-patch16-224", "google-bert/bert-base-uncased"
# # )


In [ ]:
# Đường dẫn đến thư mục chứa ảnh
image_train_dir = '/kaggle/input/dsc-dataset/train-images/train-images' 
image_test_dir = '/kaggle/input/dsc-dataset/dev-images/dev-images'

# Lấy danh sách các tệp ảnh .jpg
image_train_paths = [os.path.join(image_train_dir, file) for file in os.listdir(image_train_dir) if file.endswith('.jpg')]
image_test_paths = [os.path.join(image_test_dir, file) for file in os.listdir(image_test_dir) if file.endswith('.jpg')]

# Đọc dữ liệu từ file JSON
with open('/kaggle/input/data-demoji-remove-hashtag-url-v3/processed_train_demoji_remove_hashtag_url_v3.json', 'r', encoding='utf-8') as f:
    data_train = json.load(f)

with open('/kaggle/input/data-demoji-remove-hashtag-url-v3/processed_pred_demoji_remove_hashtag_url_v3.json', 'r', encoding='utf-8') as f:
    data_test = json.load(f)
    

combined_data_train = []
combined_data_test = []

label_mapping = {
    "not-sarcasm": 0,
    "text-sarcasm": 1,
    "image-sarcasm": 2,
    "multi-sarcasm": 3,
}

# Khởi tạo biến id để đếm số lượng mục
train_id = 0
test_id = 0

# Duyệt qua từng mục trong dữ liệu JSON để tạo combined_data_train
for key, value in data_train.items():
    image_name = value["image"]
    image_path = os.path.join(image_train_dir, image_name)  # Tạo đường dẫn tuyệt đối cho từng hình ảnh
    
    # Kiểm tra xem ảnh có trong danh sách hay không
    if image_path in image_train_paths:
        label = value.get("label")  # Lấy giá trị label
        if label is not None and label in label_mapping:  # Kiểm tra giá trị label
            combined_data_train.append({
                'id': train_id,  # Thêm id theo thứ tự
                'image': image_path,  # Lưu đường dẫn tuyệt đối
                'caption': value["caption"],
                'label': label_mapping[label],  # Lưu label từ label_mapping\
#                 'text_image': value["text_image"]
            })
            train_id += 1  # Tăng id sau mỗi lần thêm

# Duyệt qua từng mục trong dữ liệu JSON để tạo combined_data_test
for key, value in data_test.items():
    image_name = value["image"]
    image_path = os.path.join(image_test_dir, image_name)  # Tạo đường dẫn tuyệt đối cho từng hình ảnh

    # Kiểm tra xem ảnh có trong danh sách hay không
    if image_path in image_test_paths:
        combined_data_test.append({
            'id': test_id,  # Thêm id theo thứ tự
            'image': image_path,  # Lưu đường dẫn tuyệt đối
            'caption': value["caption"],
            'label': value["label"],  # Lưu label từ label_mapping
#             'text_image': value["text_image"]
        })
        test_id += 1  # Tăng id sau mỗi lần thêm

# In ra kết quả
# print(combined_data_train)
# print(combined_data_test)

# Tách caption và label từ combined_data_train
texts_train = [item['caption'] for item in combined_data_train]
labels_train = [item['label'] for item in combined_data_train]
id_train = [item['id'] for item in combined_data_train]
images_train = [item['image'] for item in combined_data_train]
# text_images_train = [item['text_image'] for item in combined_data_train]
texts_test = [item['caption'] for item in combined_data_test]
labels_test = [item['label'] for item in combined_data_test]
id_test = [item['id'] for item in combined_data_test]
images_test = [item['image'] for item in combined_data_test]
# text_image_test = [item['text_image'] for item in combined_data_test]


In [ ]:
from sklearn.model_selection import train_test_split


# Chia dữ liệu thành train và validation với tỷ lệ 80-20
texts_train, texts_val, labels_train, labels_val, id_train, id_val, images_train, images_val = train_test_split(
    texts_train, labels_train, id_train, images_train, test_size=0.2, random_state=42)

# In kết quả để kiểm tra
print(f"Số lượng dữ liệu train: {len(texts_train)}")
print(f"Số lượng dữ liệu validation: {len(texts_val)}")


In [ ]:
!pip install open_clip_torch



In [ ]:
import torch
import open_clip
import requests
from PIL import Image


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, CLIPProcessor, CLIPModel
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

# 1. Khởi tạo mô hình
phobert_model_name = "vinai/phobert-base-v2"
image_model_name = "google/vit-base-patch16-224"

phobert_tokenizer = AutoTokenizer.from_pretrained(phobert_model_name)
phobert_model = AutoModel.from_pretrained(phobert_model_name)

image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
image_model =ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [ ]:
# # vit_image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
# # processor = VisionTextDualEncoderProcessor(vit_image_processor, bert_tokenizer)

In [ ]:
def preprocess_text(texts, tokenizer):

  inputs = tokenizer(
      texts,
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256  # Điều chỉnh max_length nếu cần
  )
  return inputs.input_ids, inputs.attention_mask

In [ ]:
def preprocess_image(image_paths, processor):

    images = [Image.open(path).convert("RGB") for path in image_paths]
    images = [image.resize((224, 224)) for image in images]
    inputs = processor(
    images=images,
    return_tensors="pt",
    )
    return inputs.pixel_values

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, input_ids, attention_mask, pixel_values, labels):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.pixel_values = pixel_values
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return {
        "input_ids": self.input_ids[idx],
        "attention_mask": self.attention_mask[idx],
        "pixel_values": self.pixel_values[idx],
        "labels": self.labels[idx]
    }


In [ ]:
# Xử lý dữ liệu train
train_pixel_values = preprocess_image(images_train, image_processor)
train_input_ids, train_attention_mask = preprocess_text(texts_train, phobert_tokenizer)

# Xử lý dữ liệu val
val_input_ids, val_attention_mask = preprocess_text(texts_val, phobert_tokenizer)
val_pixel_values = preprocess_image(images_val, image_processor)

# Tạo DataLoader
train_dataset = CustomDataset(train_input_ids, train_attention_mask, train_pixel_values, labels_train)
val_dataset = CustomDataset(val_input_ids, val_attention_mask, val_pixel_values, labels_val)


In [ ]:
# Lưu dữ liệu train
torch.save({
    'input_ids': train_input_ids,
    'attention_mask': train_attention_mask,
    'pixel_values': train_pixel_values,
    'labels': labels_train
}, 'train_data.pt')

# Lưu dữ liệu val
torch.save({
    'input_ids': val_input_ids,
    'attention_mask': val_attention_mask,
    'pixel_values': val_pixel_values,
    'labels': labels_val
}, 'val_data.pt')


In [ ]:
# Tải dữ liệu train
train_data = torch.load('train_data.pt')
train_dataset = CustomDataset(
    train_data['input_ids'],
    train_data['attention_mask'],
    train_data['pixel_values'],
    train_data['labels']
)

# Tải dữ liệu val
val_data = torch.load('val_data.pt')
val_dataset = CustomDataset(
    val_data['input_ids'],
    val_data['attention_mask'],
    val_data['pixel_values'],
    val_data['labels']
)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True ,num_workers = 4)  # Điều chỉnh batch_size nếu cần
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True ,num_workers = 4)

In [ ]:
print(len(train_dataloader))

In [ ]:
print(len(set(labels_train)))

In [ ]:
# class EnsembleModel(nn.Module):
#     def __init__(self, phobert_model, image_model, num_classes):
#         super(EnsembleModel, self).__init__()
#         self.phobert_model = phobert_model
#         self.image_model = image_model
#         self.classifier = nn.Linear(
#             phobert_model.config.hidden_size + image_model.config.hidden_size, num_classes
#         )

#     def forward(self, input_ids, attention_mask, pixel_values):
#         # Lấy output từ PhoBERT
#         phobert_output = self.phobert_model(input_ids=input_ids, attention_mask=attention_mask)
#         phobert_features = phobert_output.last_hidden_state[:, 0, :]  # Đặc trưng của token [CLS]
        
#         # Lấy output từ ViT, yêu cầu output_hidden_states=True để lấy hidden states của lớp cuối
#         image_output = self.image_model(pixel_values=pixel_values, output_hidden_states=True)
#         image_features = image_output.hidden_states[-1][:, 0, :]  # Đặc trưng của token [CLS] từ lớp cuối
        
#         # Nối các đặc trưng từ PhoBERT và ViT
#         combined_features = torch.cat((phobert_features, image_features), dim=1)
#         logits = self.classifier(combined_features)
        
#         return logits


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class EnsembleModel(nn.Module):
    def __init__(self, phobert_model, image_model, num_classes, dropout_rate=0.3):
        super(EnsembleModel, self).__init__()
        self.phobert_model = phobert_model
        self.image_model = image_model
        
        # Thêm lớp Dropout để giảm overfitting
        self.dropout = nn.Dropout(dropout_rate)
        
        # Lớp phân loại
        self.classifier = nn.Linear(
            phobert_model.config.hidden_size + image_model.config.hidden_size, num_classes
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        # Lấy output từ PhoBERT
        phobert_output = self.phobert_model(input_ids=input_ids, attention_mask=attention_mask)
        phobert_features = phobert_output.last_hidden_state[:, 0, :]  # Đặc trưng của token [CLS]
        
        # Lấy output từ ViT, yêu cầu output_hidden_states=True để lấy hidden states của lớp cuối
        image_output = self.image_model(pixel_values=pixel_values, output_hidden_states=True)
        image_features = image_output.hidden_states[-1][:, 0, :]  # Đặc trưng của token [CLS] từ lớp cuối
        
        # Nối các đặc trưng từ PhoBERT và ViT
        combined_features = torch.cat((phobert_features, image_features), dim=1)
        
        # Áp dụng dropout để ngăn overfitting
        combined_features = self.dropout(combined_features)
        
        # Phân loại
        logits = self.classifier(combined_features)
        
        return logits



# Hàm huấn luyện và đánh giá sử dụng mô hình này


In [ ]:
# Sử dụng Focal Loss để giảm ảnh hưởng của mất cân bằng lớp
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss

In [ ]:
# Khởi tạo optimizer với weight decay mạnh hơn và cosine annealing
def get_optimizer_with_regularization(model, base_learning_rate=1e-5, weight_decay=1e-3, T_0=5, T_mult=2):
    optimizer = optim.AdamW(
        model.parameters(),
        lr=base_learning_rate,
        weight_decay=weight_decay
    )
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=T_mult)
    return optimizer, scheduler


In [ ]:
num_classes = len(set(labels_train))  # Đảm bảo số lớp của mô hình khớp với số lớp của nhãn

phobert_model = phobert_model.to(device)
image_model = image_model.to(device)
model = EnsembleModel(phobert_model, image_model, num_classes, dropout_rate=0.5).to(device)
# Ví dụ sử dụng model



In [ ]:
# Xác định số lượng lớp trong mô hình PhoBERT và ViT
phobert_num_layers = len(model.phobert_model.encoder.layer)
image_num_layers = len(model.image_model.vit.encoder.layer)  # Truy cập vào transformer layers của ViT


In [ ]:
print(phobert_num_layers)
print(image_num_layers)

In [ ]:
# Tinh chỉnh nửa số lớp cuối của PhoBERT
for param in model.phobert_model.parameters():
    param.requires_grad = False  # Đóng băng tất cả các lớp
for layer in model.phobert_model.encoder.layer[-(phobert_num_layers // 2):]:  # Tinh chỉnh nửa số lớp cuối
    for param in layer.parameters():
        param.requires_grad = True

# Tinh chỉnh nửa số lớp cuối của CLIP
for param in model.image_model.parameters():
    param.requires_grad = False  # Đóng băng tất cả các lớp
for layer in model.image_model.vit.encoder.layer[-(image_num_layers // 2):]:  # Tinh chỉnh nửa số lớp cuối
    for param in layer.parameters():
        param.requires_grad = True

In [ ]:
# # Kiểm tra xem các tham số của mô hình có gradient hay không
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"Tham số {name} có gradient và sẽ được tinh chỉnh.")
#     else:
#         print(f"Tham số {name} không có gradient và sẽ không được tinh chỉnh.")


In [ ]:
# pre_training_params = {name: param.detach().clone() for name, param in model.named_parameters()}


In [ ]:
# Thêm thư viện để theo dõi tiến trình
from tqdm import tqdm
from sklearn.metrics import f1_score


# Huấn luyện mô hình
num_epochs = 15  # Điều chỉnh số lượng epochs
early_stopping_patience = 3  # Số epoch không cải thiện trước khi dừng sớm
best_val_loss = float('inf')
epochs_no_improve = 0

# Chuyển các mô hình vào chế độ huấn luyện
model.train()
phobert_model.train()
image_model.train()

import torch.optim as optim


# Xác định hàm mất mát và bộ tối ưu
optimizer, scheduler = get_optimizer_with_regularization(model)

for epoch in range(num_epochs):
    # Biến để tính toán loss và accuracy của tập train
    train_loss = 0.0
    train_preds = []
    train_labels = []
    
    # Theo dõi tiến trình từng batch với tqdm
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Lan truyền thuận và ngược
        outputs = model(input_ids, attention_mask, pixel_values)
        loss = FocalLoss()(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Cộng dồn loss
        train_loss += loss.item()
        _, preds = torch.max(outputs, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

        del input_ids, attention_mask, pixel_values, labels, outputs, loss
        torch.cuda.empty_cache()

    # Tính toán metrics cho epoch
    train_accuracy = accuracy_score(train_labels, train_preds)
    val_loss = 0.0
    val_preds = []
    val_labels = []
    
    model.eval()  # Chuyển mô hình sang chế độ đánh giá
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask, pixel_values)
            loss = FocalLoss()(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            del input_ids, attention_mask, pixel_values, labels, outputs, loss
            torch.cuda.empty_cache()

    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1_macro = f1_score(val_labels, val_preds, average='macro')
    
    print(f"\nEpoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss/len(train_dataloader):.4f}")
    print(f"  Train Accuracy: {train_accuracy:.4f}")
    print(f"  Validation Loss: {val_loss/len(val_dataloader):.4f}")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Validation F1 Score (Macro): {val_f1_macro:.4f}")

    # Lưu checkpoint và kiểm tra Early Stopping
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss / len(train_dataloader),
        'train_accuracy': train_accuracy,
        'val_loss': val_loss / len(val_dataloader),
        'val_accuracy': val_accuracy,
        'val_f1_macro': val_f1_macro,
    }
    
    checkpoint_path = f"checkpoint_epoch_{epoch+1}.pth"
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved to {checkpoint_path}")
    
    # Kiểm tra điều kiện dừng sớm (Early Stopping)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print("Stopping early due to no improvement in validation loss.")
            break

    model.train()


In [ ]:


# for epoch in range(3):
#     # Biến để tính toán loss và accuracy của tập train
#     train_loss = 0.0
#     train_preds = []
#     train_labels = []
    
#     # Theo dõi tiến trình từng batch với tqdm
#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         pixel_values = batch["pixel_values"].to(device)
#         labels = batch["labels"].to(device)

#         # Lan truyền thuận và ngược
#         outputs = model(input_ids, attention_mask, pixel_values)
#         loss = criterion(outputs, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Cộng dồn loss
#         train_loss += loss.item()
#         _, preds = torch.max(outputs, dim=1)
#         train_preds.extend(preds.cpu().numpy())
#         train_labels.extend(labels.cpu().numpy())

#         del input_ids, attention_mask, pixel_values, labels, outputs, loss
#         torch.cuda.empty_cache()

#     # Tính toán metrics cho epoch
#     train_accuracy = accuracy_score(train_labels, train_preds)
#     val_loss = 0.0
#     val_preds = []
#     val_labels = []
    
#     model.eval()  # Chuyển mô hình sang chế độ đánh giá
#     with torch.no_grad():
#         for batch in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             pixel_values = batch["pixel_values"].to(device)
#             labels = batch["labels"].to(device)

#             outputs = model(input_ids, attention_mask, pixel_values)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()

#             _, preds = torch.max(outputs, dim=1)
#             val_preds.extend(preds.cpu().numpy())
#             val_labels.extend(labels.cpu().numpy())

#             del input_ids, attention_mask, pixel_values, labels, outputs, loss
#             torch.cuda.empty_cache()

#     val_accuracy = accuracy_score(val_labels, val_preds)
#     val_f1_macro = f1_score(val_labels, val_preds, average='macro')
    
#     print(f"\nEpoch {epoch+1}/{num_epochs}:")
#     print(f"  Train Loss: {train_loss/len(train_dataloader):.4f}")
#     print(f"  Train Accuracy: {train_accuracy:.4f}")
#     print(f"  Validation Loss: {val_loss/len(val_dataloader):.4f}")
#     print(f"  Validation Accuracy: {val_accuracy:.4f}")
#     print(f"  Validation F1 Score (Macro): {val_f1_macro:.4f}")

#     # Lưu checkpoint và kiểm tra Early Stopping
#     checkpoint = {
#         'epoch': epoch + 1,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'train_loss': train_loss / len(train_dataloader),
#         'train_accuracy': train_accuracy,
#         'val_loss': val_loss / len(val_dataloader),
#         'val_accuracy': val_accuracy,
#         'val_f1_macro': val_f1_macro,
#     }
    
#     checkpoint_path = f"checkpoint_epoch_{epoch+1}.pth"
#     torch.save(checkpoint, checkpoint_path)
#     print(f"Checkpoint saved to {checkpoint_path}")
    
#     # Kiểm tra điều kiện dừng sớm (Early Stopping)
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         epochs_no_improve = 0
#     else:
#         epochs_no_improve += 1
#         if epochs_no_improve >= early_stopping_patience:
#             print("Stopping early due to no improvement in validation loss.")
#             break

#     model.train()


In [ ]:
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader

# Đảm bảo mô hình đang ở chế độ đánh giá (evaluation mode)
model.eval()

# Tạo DataLoader cho tập validation
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Biến lưu trữ nhãn thực tế và dự đoán
all_labels = []
all_preds = []

# Không tính gradient khi đánh giá
with torch.no_grad():
    for batch in val_dataloader:
        # Lấy dữ liệu và chuyển sang GPU nếu có
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        # Tiến hành dự đoán
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        
        # Kiểm tra đầu ra để lấy logits
        if isinstance(outputs, tuple):
            logits = outputs[0]  # Trường hợp mô hình trả về một tuple (logits, các giá trị khác)
        else:
            logits = outputs  # Trường hợp mô hình chỉ trả về logits

        # Chuyển logits thành dự đoán (giả sử sử dụng softmax để lấy xác suất, sau đó chọn lớp có xác suất cao nhất)
        preds = torch.argmax(logits, dim=-1)
        
        # Lưu trữ kết quả
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# In ra Classification Report
report = classification_report(all_labels, all_preds, target_names=['not-sarcasm', 'text-sarcasm', 'image-sarcasm', 'multi-sarcasm'])
print("Classification Report:\n", report)


In [ ]:
# import torch

# # Đường dẫn đến checkpoint
# checkpoint_path = '/kaggle/working/checkpoint_epoch_5.pth'

# # Tải checkpoint từ file
# checkpoint = torch.load(checkpoint_path)

# # Lấy trạng thái mô hình và các tham số huấn luyện tại epoch 5
# epoch = checkpoint['epoch']
# model_state_dict = checkpoint['model_state_dict']
# optimizer_state_dict = checkpoint['optimizer_state_dict']
# train_loss = checkpoint['train_loss']
# train_accuracy = checkpoint['train_accuracy']
# val_loss = checkpoint['val_loss']
# val_accuracy = checkpoint['val_accuracy']
# val_f1_macro = checkpoint['val_f1_macro']

# # In ra thông tin tại epoch 5
# print(f"Epoch: {epoch}")
# print(f"Train Loss: {train_loss}")
# print(f"Train Accuracy: {train_accuracy}")
# print(f"Validation Loss: {val_loss}")
# print(f"Validation Accuracy: {val_accuracy}")
# print(f"Validation F1 Score (Macro): {val_f1_macro}")

# # Khôi phục trạng thái mô hình và optimizer
# model.load_state_dict(model_state_dict)
# optimizer.load_state_dict(optimizer_state_dict)

# # Tiếp tục huấn luyện hoặc thực hiện các tác vụ khác


In [ ]:
# import torch
# from sklearn.metrics import classification_report
# from torch.utils.data import DataLoader

# # Đảm bảo mô hình đang ở chế độ đánh giá (evaluation mode)
# model.eval()

# # Tạo DataLoader cho tập validation
# # val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# # Biến lưu trữ nhãn thực tế và dự đoán
# all_labels = []
# all_preds = []

# # Không tính gradient khi đánh giá
# with torch.no_grad():
#     for batch in val_dataloader:
#         # Lấy dữ liệu và chuyển sang GPU nếu có
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         pixel_values = batch['pixel_values'].to(device)
#         labels = batch['labels'].to(device)

#         # Tiến hành dự đoán
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values)
        
#         # Kiểm tra đầu ra để lấy logits
#         if isinstance(outputs, tuple):
#             logits = outputs[0]  # Trường hợp mô hình trả về một tuple (logits, các giá trị khác)
#         else:
#             logits = outputs  # Trường hợp mô hình chỉ trả về logits

#         # Chuyển logits thành dự đoán (giả sử sử dụng softmax để lấy xác suất, sau đó chọn lớp có xác suất cao nhất)
#         preds = torch.argmax(logits, dim=-1)
        
#         # Lưu trữ kết quả
#         all_labels.extend(labels.cpu().numpy())
#         all_preds.extend(preds.cpu().numpy())

# # In ra Classification Report
# report = classification_report(all_labels, all_preds, target_names=['not-sarcasm', 'text-sarcasm', 'image-sarcasm', 'multi-sarcasm'])
# print("Classification Report:\n", report)
